## Base Cloud ServiceHandler

In [1]:
#| default_exp services.base

In [2]:
#| export

from dc_logger.client.base import LogEntry
from dc_logger.client.exceptions import LogWriteError

In [3]:
#| exporti
from dc_logger.client.base import ServiceHandler

from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List

In [ ]:
#| export

@dataclass
class CloudHandler(ServiceHandler):
    """Base class for cloud log handlers"""

    # def __init__(self, config):
    #     super().__init__(config)
    #     self.cloud_config = config.to_platform_config()
    #     config.validate_config()

    @abstractmethod
    async def _send_to_cloud(self, entries: List[LogEntry]) -> bool:
        """Send entries to cloud provider"""
        pass

    async def write(self, entries: List[LogEntry]) -> bool:
        """Write entries to cloud provider"""
        try:
            return await self._send_to_cloud(entries)
        except Exception as e:
            raise LogWriteError(f"Error sending logs to cloud provider: {e}")


In [ ]:
#| export

class ConsoleHandler(ServiceHandler):
    """Handler for console output"""

    async def _write_json(self, entry: LogEntry) -> None:
        pass

    async def _write_text(self, entry: LogEntry) -> None:
        pass

    async def write(self, entries: List[LogEntry]) -> bool:
        """Write entries to console"""
        try:
            for entry in entries:
                if self.config.format == "json":
                    if self.config.pretty_print:
                        # Pretty print JSON for development
                        print(json.dumps(entry.to_dict(), indent=2, default=str))
                        print("-" * 80)  # Separator for readability
                    else:
                        print(entry.to_json())
                else:
                    print(
                        f"[{entry.timestamp}] {entry.level.value} {entry.logger}: {entry.message}"
                    )
            return True
        except Exception as e:
            print(f"Error writing to console: {e}")
            return False

    async def flush(self) -> bool:
        """Console output doesn't need flushing"""
        return True


In [5]:
#| hide
import nbdev; nbdev.nbdev_export('./base.ipynb')